In [1]:
import json
import requests
import asyncio
import random
import pandas as pd
from rouge import Rouge
import nltk
import nest_asyncio
import time
import aiohttp  

nest_asyncio.apply()
dataset_list = []

with open("development.json", "r") as file:
    seed_data = json.load(file)

with open("shopping_chatglm_data.json", "r") as file:
    dataset_list = json.load(file)
data_str = [item["text"] for item in dataset_list]
df_seed = pd.DataFrame(seed_data)

def split_string(gpt_output: str):
    parts = gpt_output.split('###')
    return parts

def delete_repeat(threshold, dataset_list: list, tar):
    if len(dataset_list) == 0:
        return True
    for item in dataset_list:
        rouge = Rouge()
        scores = rouge.get_scores([item], [tar])
        if scores[0]['rouge-l']['f'] > threshold:
            return False
    return True

In [2]:
flag = 0
final_seed_len = 5
generate_time = 1
num_list = list(range(len(df_seed)))
selected_set = []
prefix_prompt = """
You are asked to come up with a set of 5 diverse tasks. These task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.

Here are the requirements:
0. The theme of this task is about Shopping Concept Understanding, aims to evaluate the model's ability to understand shopping entities in the form of texts, such as product names, product categories, attributes, product descriptions, reviews, etc
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The way of expressing words in the input_field needs to be diversified.
3. In order to make the type of task be diverse, trying to generate task types different from the examples. The list should include diverse types of tasks like open-ended generation, classification, ranking, multi-choice etc.
4. A GPT language model should be able to complete the output_field based on the input_field. 
5. The output_field should be an appropriate and correct response to the input_field. Make sure the output_field is less than 100 words.
6. Each task needs to be enclosed with “###” just like the examples. Note that the task type should be at the beginning of the example.
7. Input_field must end with 'Answer ' or 'Output: '. Make sure the input_field is more than 200 words!!!
List of 5 tasks:
"""

In [3]:
prompt_gpt_input_list = []

for i in range(generate_time):
     while True:
        selected_numbers = set(random.sample(num_list, final_seed_len))
        if selected_numbers not in selected_set:
            selected_set.append(selected_numbers)
            break

for set_item in selected_set[-generate_time:]:
    df_final_seed = pd.DataFrame([], columns = ["input_field", "output_field", "task_name", "task_type", "metric", "is_multiple_choice"])
    item_list = ""
    for j in set_item:
        df_final_seed.loc[len(df_final_seed)] = df_seed.loc[j]
    for j in range(len(df_final_seed)):
        temp_string = '\n###\n task_type:' + df_final_seed['task_type'][j] + "\n input_field: " + str(df_final_seed['input_field'][j]) + '\n output_field: ' + str(df_final_seed['output_field'][j]) + '\n'
        item_list += temp_string
    prompt_gpt_input = prefix_prompt + item_list
    prompt_gpt_input_list.append(prompt_gpt_input)

In [4]:
from http import HTTPStatus
import dashscope
import asyncio
dashscope.api_key="###"

async def call_with_messages(msg):
    messages = [
        {'role': 'user', 'content': msg}]
    response = dashscope.Generation.call(
        'qwen-long',
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        return response
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        return 0

In [5]:
async def main():   
    msgs =  prompt_gpt_input_list
    # 使用asyncio.gather来并行执行多个fetch_data函数  
    tasks = [call_with_messages(msg) for msg in msgs]   
    results = await asyncio.gather(*tasks)  
    print("Result accepted")
    for result in results:  
        if result == 0:
            continue
        string = result["output"]["choices"][0]["message"]["content"]
        tar_str_list = split_string(string)
        for tar in tar_str_list:
            if tar != "" and delete_repeat(0.7, data_str, tar):
                dataset_list.append({"text": tar})
                data_str.append(tar)
await main()

Result accepted


In [6]:
json_data = json.dumps(dataset_list, indent=4, separators=(',', ': '))
filename = 'shopping_chatglm_data.json'

# 将JSON字符串写入文件
with open(filename, 'w') as f:  
    f.write(json_data)

In [7]:
dataset_list

[{'text': ' task_type:classification\nInput: Classify the product category for the following product description: "The XYZ blender features a powerful 1200-watt motor, multiple speed settings, and a large 64-ounce BPA-free pitcher, perfect for making smoothies, soups, and sauces."\nOutput: Blender\n\n'},
 {'text': " task_type:open-ended\nInput: Output a detailed review for a customer who recently purchased a pair of running shoes with excellent shock absorption and breathable mesh material.\nOutput: The XYZ running shoes have been a game-changer for my workouts! Their exceptional shock absorption has significantly reduced foot and joint fatigue during my runs. The breathable mesh material keeps my feet cool and dry, even during intense sessions, preventing any unpleasant odors or discomfort. The design is stylish and fits true to size. I appreciate the secure grip and stability they provide on various surfaces. Overall, I'm extremely satisfied with this purchase and would highly recomm